## Import Libraries

In [16]:
import matplotlib.pyplot as plt
import urllib
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import os,sys, shutil
import time
from datetime import date
try:
    import cPickle as pickle
except:
    import pickle
import pprint
from collections import deque
from shutil import copyfile
import random
import glob
# Import the required modules
import cv2, os
import numpy as np
from PIL import Image
from sklearn.cross_validation import KFold
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# Logistic Regression
from sklearn import datasets
from sklearn import metrics

In [2]:
pkl_fl = open("linkedin_profiles.pickle","rb")
my_original_list=pickle.load(pkl_fl) # errors out here
pkl_fl.close()

In [3]:
my_original_list[0]

{'Full_Name': 'Eryn Olson',
 'Gender': 'Female',
 'Profile_Url': 'https://media.licdn.com/mpr/mpr/shrinknp_400_400/AAEAAQAAAAAAAAbyAAAAJDMzZjQ4NjViLTIwZTctNDM0Yi05ODI5LTk0NmYzZWZmZDI5Yg.jpg',
 'Recommended_Ids': ['erynolson',
  'eryn-olson-62639510a',
  'eryn-olson-432679b1',
  'eryn-olson-1351aa74',
  'courtney-tillman-8b62a664',
  'chance-cozby-13235224',
  'bill-gates-b1a606b0',
  'sarah-eves-2937a719',
  'andrew-solheim-810a3517',
  'andrea-cundiff-76020535',
  'mollie-harper-6b850947',
  'tyler-shaw-934b7',
  'donna-conroy-66987340',
  'paul-wood-73455920'],
 'User_ID': 'eryn-olson-50328143',
 'age': 20}

## label the Images and store into respective folders

In [26]:
directory2 = "Label_Images_Age"

if os.path.exists(directory2):
    shutil.rmtree(directory2)
    os.makedirs(directory2)  
else:
    os.makedirs(directory2)   
    
fileList = glob.glob("./Images/*.*")

for id,fp in enumerate(fileList):
    filename, file_extension = os.path.splitext(fp)
    uid = filename.split('/')[-1]
    
    for prof in my_original_list:
        if prof['User_ID'] == uid:
            
            prof_age = prof['age']
            
            if (0 <= prof_age <= 30):
                new_file_extension = 'Adult'
            else:
                new_file_extension = 'Senior'
            
            copyfile(filename + ".jpg", './Label_Images_Age/'+ uid + '.' + str(id) + "." + new_file_extension +'.jpg')
            #copyfile(filename + ".jpg", new_file_extension +'/' + filename + ".jpg")

In [27]:
# For face detection we will use the Haar Cascade provided by OpenCV.
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)

# For face recognition we will the the LBPH Face Recognizer 
recognizer = cv2.createLBPHFaceRecognizer()

In [29]:
def get_images_and_labels(path):
    # Append all the absolute image paths in a list image_paths
    
    image_paths = [os.path.join(path, f) for f in os.listdir(path)]
    # images will contains face images
    images = []
    # labels will contains the label that is assigned to the image
    labels = []
    #gender will contains 1 or 0 indecating male or female
    age =[]
    
    for image_path in image_paths:
        # Read the image and convert to grayscale
        try:
            image_pil = Image.open(image_path).convert('L')
            # Convert the image format into numpy array
            image = np.array(image_pil, 'uint8')
            # Get the label of the image
        except:
            pass
        
        nbr = int(os.path.split(image_path)[1].split(".")[1])
        age_current = os.path.split(image_path)[1].split(".")[2]
        print nbr
        
        # Detect the face in the image
        faces = faceCascade.detectMultiScale(image)
        # If face is detected, append the face to images and the label to labels
        try:
            for (x, y, w, h) in faces:

                ref_image = image[y: y + h, x: x + w]
                resized = cv2.resize(ref_image, (100, 100), interpolation = cv2.INTER_AREA)
                images.append(resized)
                labels.append(nbr)

                #if age_current == 'Child':
                #    age.append(0)
                if age_current == 'Adult':
                    age.append(0)
                else:
                    age.append(1)
                    
                cv2.imshow("Adding faces to traning set...", resized)
                cv2.waitKey(10)
        except:
            pass
    # return the images list and labels list
    print "lables"
    print labels
    print "gender_current"
    print age
    
    return images, labels, age

In [30]:
images, labels, ageList = get_images_and_labels('Label_Images_Age')

cv2.destroyAllWindows()

456
421
858
1918
244
2151
623
1253
1234
735
1892
1504
947
2134
867
1585
1822
553
792
138
1189
198
1045
175
1103
1429
1444
33
351
2024
1681
662
1565
448
1056
61
2219
715
1876
2090
1425
1513
1597
113
2225
413
1962
502
743
2201
2154
1138
2100
1406
2058
207
885
736
1021
1210
112
144
1225
157
2049
1851
1463
1201
606
106
2020
1366
2047
2003
1684
1802
420
194
1996
449
520
950
1145
380
567
787
1386
221
159
809
2156
1586
1148
2210
685
2054
2205
1886
615
702
695
1908
686
1405
63
670
1525
1071
873
672
2164
775
1306
1509
2007
1378
558
2114
1929
1188
1466
658
1905
1127
1931
437
2041
445
1645
1980
1100
1239
2138
2213
1975
2071
1620
911
872
340
1532
1327
513
256
1643
67
1496
1922
1678
1033
1967
661
952
1800
1462
210
2176
441
1479
158
1322
544
1361
796
912
1495
1698
2061
1334
334
1083
744
1979
1480
2042
1835
1839
2069
605
1952
614
537
163
949
1451
1959
1369
227
2165
2206
574
199
462
1960
173
1433
243
1877
826
1307
321
2157
781
959
1541
2217
577
1183
1561
1769
2044
1505
2147
92
1764
849
1501
1066
1285


In [31]:
images[0].shape

(100, 100)

## Making the Dataset 

In [32]:
res_images = []
res_age = []

for img in images:
    res_images.append(img.reshape(1,10000))    # (100*100 image size)

for age in ageList:
    res_age.append(np.array(age).reshape(1,))
    
res_images = np.concatenate(res_images, axis=0)
res_age = np.concatenate(res_age,axis=0)

In [35]:
X,y = res_images,res_age

In [39]:
X.shape

(1810, 10000)

In [41]:
y.shape

(1810,)

## Test the Model for Age

In [37]:
kf = KFold(X.shape[0], n_folds=10, shuffle=True)
accuracy = 0.0
precision = [0, 0]
recall = [0, 0]
f_measure = [0, 0]

lr = LogisticRegression()
inbuilt_accuracy_score = 0.0

for test, train in kf:
    X_Train, X_Test = X[train], X[test]
    Y_Train, Y_Test = y[train], y[test]

    lr.fit(X_Train, Y_Train)

    

    pred = lr.predict(X_Test)

    inbuilt_accuracy_score += accuracy_score(Y_Test, (pred))

#precision = map(lambda x: x / 10, precision)
#recall = map(lambda x: x / 10, recall)
#f_measure = map(lambda x: x / 10, f_measure)
#accuracy /= 10
#print "Confusion Matrix : \n\n", cm

inbuilt_accuracy_score /= 10

#print "*" * 100
#print "\t\t\t Inbuilt Method"

print "\nAccuracy from inbuilt Method: ", inbuilt_accuracy_score
print(metrics.classification_report(Y_Test, pred))
print(metrics.confusion_matrix(Y_Test, pred))




Accuracy from inbuilt Method:  0.585389809699
             precision    recall  f1-score   support

          0       0.69      0.65      0.67      1055
          1       0.42      0.46      0.44       574

avg / total       0.60      0.59      0.59      1629

[[689 366]
 [309 265]]


## Histogram of Ages

In [42]:
#!/usr/bin/env python
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

#mu, sigma = 100, 15
#x = mu + sigma*np.random.randn(10000)
ages = [prof['age'] for prof in my_original_list]
x= np.array(ages) 
# the histogram of the data
n, bins, patches = plt.hist(x, 50, normed=1, facecolor='blue', alpha=0.75)

# add a 'best fit' line
#y = mlab.normpdf(bins, mu, sigma)
#l = plt.plot(bins, y, 'r--', linewidth=1)

plt.xlabel('Age')
plt.ylabel('Probability')
plt.title(r'$Histogram\ \ of\ \ Ages$')
plt.axis([1, 80, 0, 0.09])
plt.grid(True)

plt.show()